<a href="https://colab.research.google.com/github/kevinscaria/InstructABSA/blob/main/ATE_Training_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

In [1]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  #!pip install sentencepiece

NameError: name 'IN_COLAB' is not defined

In [ ]:
import os
import torch

if IN_COLAB:
    root_path = 'Enter drive path'
else:
    root_path = 'Enter local path'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install --upgrade torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 17.5 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 14.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:00:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 893.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 

In [2]:
import torch
print(torch.__version__)


1.13.1+cu117


In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

## Training

In [2]:
import os
task_name = 'ate'
experiment_name = 'lapt2014_iabsa1'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  lapt2014_iabsa1
Model output path:  ./Models/ate/allenaitk-instruct-base-def-pos-lapt2014_iabsa1


In [3]:
from sklearn.model_selection import train_test_split
import ast
# Load the data
id_data_file_path = '/home/ec2-user/SageMaker/InstructABSA-main/Dataset/senti.csv'
id_df = pd.read_csv(id_data_file_path)


#id_df['Decisions'] = id_df['Decisions'].apply(ast.literal_eval)  # Convert string representation of dictionaries into actual dictionaries
#id_df['labels'] = id_df['Decisions'].apply(lambda x: ', '.join([f"{k}: {v}" for k, v in x.items()]))  # Adjusted lambda function




In [4]:

def is_valid_dict(s):
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return True
        else:
            return False
    except (ValueError, SyntaxError):
        return False

invalid_dict_rows = id_df['Decisions'].apply(lambda x: not is_valid_dict(x))
print(f"Invalid rows: {invalid_dict_rows.sum()}")

if invalid_dict_rows.sum() > 0:
    print("Displaying some invalid rows:")
    print(id_df.loc[invalid_dict_rows, 'Decisions'])


Invalid rows: 0


In [5]:
type(id_df['Decisions'].iloc[0])
print(id_df['Decisions'].apply(type).value_counts())


<class 'str'>    10753
Name: Decisions, dtype: int64


In [6]:
type(id_df['Decisions'].iloc[0])
print(id_df['Decisions'].apply(type).value_counts())
import numpy as np
def try_eval(s):
    if isinstance(s, dict):
        return s
    else:
        try:
            s = s.replace("'", '"')  # replace single quotes with double quotes
            #print(type(s))
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            #print(type(s))
            return np.nan  # or some value to indicate a problem


<class 'str'>    10753
Name: Decisions, dtype: int64


In [7]:
# Convert string representation of dictionaries into actual dictionaries
#id_df['Decisions'] = id_df['Decisions'].apply(ast.literal_eval)
import ast
# Split the data into train and test datasets
id_train_df, id_test_df = train_test_split(id_df, test_size=0.2, random_state=42)  # 80% for training, 20% for testing



id_train_df['Decisions'] = id_train_df['Decisions'].apply(try_eval)
id_test_df['Decisions'] = id_test_df['Decisions'].apply(try_eval)

In [8]:


# Check and see how many problematic rows there are
#print(id_train_df['Decisions'].isna().sum())
#print(id_test_df['Decisions'].isna().sum())
# Print some problematic rows
#print(id_train_df[id_train_df['Decisions'].isna()]['Decisions'].head(10))

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_train_df, id_test_df)

if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id,'Words', 'Title', 'Decisions',  instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'Words', 'Title', 'Decisions', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])


In [9]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)
use_mps = False  # or False, depending on your requirement

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':32,
    'num_train_epochs':1,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495M [00:00<?, ?B/s]

Map:   0%|          | 0/8602 [00:00<?, ? examples/s]

Map:   0%|          | 0/2151 [00:00<?, ? examples/s]

In [10]:
import torch
print(torch.__version__)


1.13.1+cu117


In [ ]:
# Train model
#!pip install --upgrade torch

#!pip install torchvision torcheia --upgrade

model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: S No., Title, text, Decisions, __index_level_0__, Words. If S No., Title, text, Decisions, __index_level_0__, Words are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

***** Running training *****
  Num examples = 8602
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 538
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


## Inference

In [ ]:
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

In [ ]:
# Convert string representation of dictionaries into actual dictionaries
#id_df['Decisions'] = id_df['Decisions'].apply(ast.literal_eval)
import ast
# Split the data into train and test datasets
id_train_df, id_test_df = train_test_split(id_df, test_size=0.2, random_state=42)  # 80% for training, 20% for testing



id_train_df['Decisions'] = id_train_df['Decisions'].apply(try_eval)
id_test_df['Decisions'] = id_test_df['Decisions'].apply(try_eval)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_train_df, id_test_df)

if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id,'Words', 'Title', 'Decisions',  instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'Words', 'Title', 'Decisions', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])


In [ ]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)
if not torch.cuda.is_available():
    t5_exp.device = 'cpu'

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

In [ ]:
p, r, f1,_ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1,_ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

In [18]:
import torch
print(torch.cuda.is_available())


False
